In [6]:
# !pip install openpyxl
# !pip install regex
import pandas as pd
import numpy as np
import datetime as dt
import regex as re
today = dt.date.today()
current_date = re.sub('-', '', str(today))[2:]

def save_xls(dict_df, path):
    """
    Save a dictionary of dataframes to an excel file, with each dataframe as a separate page
    """
    if os.path.exists(path):
        writer = pd.ExcelWriter(path, engine='openpyxl', mode ='a', if_sheet_exists='overlay')
    else:
        writer = pd.ExcelWriter(path, engine='openpyxl', mode='w' )
    for key in dict_df.keys():
        dict_df[key].to_excel(writer, key, index=False)

    writer.save()
    writer.close()

import os
os.makedirs('data/02_intermediate/', exist_ok=True)
os.makedirs('data/01_raw/', exist_ok=True)

In [2]:
raw_data = pd.read_excel('data/01_raw/SampleData.xls', skiprows=8) # Change the path of your raw file here.

start_idx = raw_data.columns.get_indexer(['Version'])[0]
end_idx = len(raw_data.columns)
sliced_variables = raw_data.iloc[:, start_idx]

# column names 
id_cols = raw_data.iloc[3, :10]

# remove first n-rows 
raw_data = raw_data.iloc[3:, :]
raw_data.columns.values[0:10] = id_cols
raw_data.columns = raw_data.columns.str.replace(' ', '') # replace spaces in column names
raw_data = raw_data.loc[~(raw_data['SampleID'] == 'Sample ID'), :] # remove first row. 
raw_data = raw_data.rename(columns={'Unnamed:10': 'date_time'}) # rename to datetime
raw_data['date_time'] = pd.to_datetime(raw_data['date_time'])
# drop columns where NA
raw_data = raw_data.dropna(axis=1, how = 'all')
raw_data.shape

(619, 105)

In [8]:
raw_data = pd.read_excel('data/01_raw/SampleDataToExcel_kor.hongwee_Feb-19-2022-01-14-07.xls', skiprows=8)

start_idx = raw_data.columns.get_indexer(['Version'])[0]
end_idx = len(raw_data.columns)
sliced_variables = raw_data.iloc[:, start_idx]

# column names 
id_cols = raw_data.iloc[3, :10]

# remove first n-rows 
raw_data = raw_data.iloc[3:, :]
raw_data.columns.values[0:10] = id_cols
raw_data.columns = raw_data.columns.str.replace(' ', '') # replace spaces in column names
raw_data = raw_data.loc[~(raw_data['SampleID'] == 'Sample ID'), :] # remove first row. 
raw_data = raw_data.rename(columns={'Unnamed:10': 'date_time'}) # rename to datetime
raw_data['date_time'] = pd.to_datetime(raw_data['date_time'])
# drop columns where NA
raw_data = raw_data.dropna(axis=1, how = 'all')
raw_data.shape

(482, 122)

In [9]:
clean_data = raw_data.copy()

In [10]:
# sort by SampleID, date_time
clean_data = clean_data.sort_values(by = ['SampleID', 'SamplePoint', 'date_time'])
unique_samples = clean_data['SamplePoint'].dropna().unique().tolist()
# Initiate a dictionary to store {'sampleID': pd.DataFrame}
sampleList = dict()
# Loop over all Sample Points, remove Na-values.
for sample in unique_samples:
    one_sample_df = clean_data.loc[clean_data['SamplePoint'] == sample]
    one_sample_df = one_sample_df.dropna(axis=1, how='all')
    # Rename columns .[d+]
    renamed = one_sample_df.columns
    renamed2 = [re.sub(r'\.[0-9]*', '', x) if '.' in x else x for x in renamed]
    one_sample_df.columns = renamed2
    if '/' in sample: # rename the values for proper saving
        sample = sample.replace('/', '_')
        print (f"Renaming samples to {sample}")
        sampleList[sample] = one_sample_df
    else:
        sampleList[sample] = one_sample_df
    

Renaming samples to F101S_O
Renaming samples to F201S_O
Renaming samples to E304 I_C


In [11]:
save_xls(sampleList, f'data/02_intermediate/Sample_{current_date}.xlsx')